# Project_hongo

## Introducción

### 1- Trabajar los datos
transformar los datos de ingreso, a variables de ingreso.

### 2- Diseño de la red
Pensar en la estructura.

### 3- Costo
Definir la función de costo para entrenar la red.

### 4- Definir el entrenamiento
Manera a modificar los parametros (pesos)

### 5- Prueba
entrenamiento y testeo.


# Imports

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import tensorflow as tf

/home/pablo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Load data

In [2]:
data = pd.read_csv("mushrooms.csv")

# Define variables

In [3]:
planedata = []
all_possible_values = []
planedata=np.reshape(np.array(planedata),(len(data),0))

num_values = 0

# Load data in planedata variable

In [4]:
for atr in data.columns:

    counter = Counter(data[atr])
    parcial_matrix = np.zeros((len(data),len(counter)))

    possible_values_parcial = list(counter.keys())
    all_possible_values += possible_values_parcial

    values = list(data[atr])

    n_reg=0
    for reg in values:
        index = possible_values_parcial.index(reg)
        parcial_matrix[n_reg,index] = 1
        n_reg += 1

    planedata = np.append(planedata,parcial_matrix,axis=1)

In [5]:
labels_data=planedata[:,0:1]
inputs_data=planedata[:,1:len(planedata[0])]

# Variable definition

In [6]:
l = 0.01
epoch = 100
batch_size = 20
n_train = int(len(labels_data)*.9)

# Divide data in inputs and test

In [7]:
labels_train=labels_data[0:n_train]
labels_test=labels_data[n_train+1:len(labels_data)]

inputs_train=inputs_data[0:n_train]
inputs_test=inputs_data[n_train+1:len(inputs_data),:]

list_labels_train = []
list_inputs_train = []

num_data = int((len(labels_train)/batch_size))

for i in range (num_data):
    list_inputs_train.append(inputs_train[i*batch_size:(i+1)*batch_size])
    list_labels_train.append(labels_train[i*batch_size:(i+1)*batch_size])

# Initializater Tensor Flow

In [8]:
# Inputs
inputs = tf.placeholder(shape=(None,len(inputs_train[0])), dtype=tf.float32, name="inputs")

# Labels
labels = tf.placeholder(shape=(None,1), dtype=tf.float32, name="labels")

In [9]:
# Initializater Tensor Flow
init = tf.global_variables_initializer()

# Building the model
input_layer = tf.contrib.layers.fully_connected(inputs=inputs, num_outputs = 100, activation_fn = tf.nn.sigmoid)
hidden1_layer = tf.contrib.layers.fully_connected(inputs=input_layer, num_outputs = 25, activation_fn = tf.nn.sigmoid)
hidden2_layer = tf.contrib.layers.fully_connected(inputs=hidden1_layer, num_outputs = 5, activation_fn = tf.nn.sigmoid)
outputs = tf.contrib.layers.fully_connected(inputs=hidden2_layer, num_outputs = 1, activation_fn = tf.nn.sigmoid)


# Cost and Optimizer

In [10]:
with tf.name_scope("Cost_and_Optimizer"):
    cost = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=outputs, name="cost"))
    optimizer = tf.train.GradientDescentOptimizer(l)
    train = optimizer.minimize(cost)


# Train neural network

In [11]:
tf.summary.scalar("Cost", cost)

with tf.Session() as sess:
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./summaries/lamnda {} epoch {} batch {}".format(l,epoch,batch_size))
    writer.add_graph(sess.graph)
    sess.run(init)

    for i in range(epoch):
        for j in range(num_data):
            sess.run(train, feed_dict={inputs: list_inputs_train[j], labels: list_labels_train[j]})

    #To calcule acurracy and cost
    if i%10:
        partial_cost = sess.run(cost, feed_dict={inputs: inputs_train, labels: labels_train})
        outputs = sess.run(output, feed_dict={inputs: inputs_train, labels: labels_train})
        predictions = outputs > 0.5

        accuracy = np.sum(labels_train == predictions) / len(labels_train)

        print("Train: Epoch N°: {} - Cost = {} - Accuracy = {}%".format(i, partial_cost, accuracy * 100))

    s = sess.run(merged_summary, feed_dict={inputs: inputs_train, labels: labels_train})
    writer.add_summary(s, i)

    # Print cost test and final acurracy
    final_cost = sess.run(cost, feed_dict={inputs: inputs_test, labels: labels_test})
    outputs = sess.run(output, feed_dict={inputs: inputs_test, labels: labels_test})
    predictions = outputs > 0.5

    final_accuracy = np.sum(labels_test == predictions) / len(labels_test)

print("Test: Cost = {} - Accuracy = {}%".format(final_cost, final_accuracy* 100))

FailedPreconditionError: Attempting to use uninitialized value fully_connected_3/biases
	 [[Node: fully_connected_3/biases/read = Identity[T=DT_FLOAT, _class=["loc:@Cost_...entDescent"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](fully_connected_3/biases)]]

Caused by op 'fully_connected_3/biases/read', defined at:
  File "/home/pablo/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/pablo/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/pablo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/pablo/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/pablo/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/home/pablo/anaconda3/lib/python3.6/site-packa